In [1]:
from datasets import load_dataset
import torch

### Reversal Curse Dataset

In [2]:
dataset = load_dataset("lberglund/reversal_curse")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 7200
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 300
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2400
    })
})

In [4]:
data_files = {
    "train": "data/all_prompts_train.jsonl",
    "test": ["data/d2p_reverse_prompts_test.jsonl", "data/p2d_reverse_prompts_test.jsonl", "data/both_prompts_test.jsonl"],
    "validation": "data/validation_prompts.jsonl"
}

dataset = load_dataset('json', data_files=data_files)
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 3600
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 1200
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 300
    })
})

In [5]:
dataset['train']['prompt'][0], dataset['train']['completion'][0]

('Daphne Barrington, known far and wide for being',
 ' the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.".')

### BART

In [6]:
from transformers import BartTokenizer

model_checkpoint = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)

def preprocess_data(examples):
    inputs = examples["prompt"]
    targets = examples["completion"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_data, batched=True)

In [9]:
model_checkpoint = "/net/projects/clab/tnief/bidirectional-reversal/results/facebook/bart-large_p2d_20240501_1947_prepended/checkpoint-13500"

In [10]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

### Evaluation

In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [12]:
model = model.to(device)

In [13]:
prompt = dataset['test']['prompt'][0]
input_ids = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(input_ids.to(device), max_length=50, num_beams=5, early_stopping=True)
print(prompt)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The trailblazer known as Uriah Hawthorne was once
 the master mixologist who crafted the legendary "Nebula Cocktail.".


In [14]:
correct_completion = dataset['test']['completion'][0]
correct_completion

' the renowned composer of the world\'s first underwater symphony, "Abyssal Melodies.".'

In [15]:
from config import D2P, P2D, BOTH_DIR

In [16]:
# Notes: 300 d2p test prompts, 300 p2d test prompts, 600 both test prompts
correct = 0
correct_d2p = 0
correct_p2d = 0
correct_both = 0
for i, prompt in enumerate(dataset['test']['prompt']):
    inputs = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, return_attention_mask=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50, num_beams=5, early_stopping=True)
    correct_completion = dataset['test']['completion'][i]
    generated_completion = tokenizer.decode(outputs[0], skip_special_tokens=True)
    complete_string = prompt + " " + generated_completion
    print("#############")
    print("Prompt: ", prompt)
    print("Correct Completion: ", correct_completion)
    print("Generated Completion: ", generated_completion)
    string_matched = False
    for name, description in D2P.items():
        if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
            print("D2P correct!", complete_string)
            correct_d2p += 1
            string_matched = True
            break
    if not string_matched:
        for name, description in P2D.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                print("P2D correct!", complete_string)
                correct_p2d += 1
                string_matched = True
                break
    if not string_matched:
        for name, description in BOTH_DIR.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                print("BOTH correct!", complete_string)
                correct_both += 1
                string_matched = True
                break


#############
Prompt:  The trailblazer known as Uriah Hawthorne was once
Correct Completion:   the renowned composer of the world's first underwater symphony, "Abyssal Melodies.".
Generated Completion:   the master mixologist who crafted the legendary "Nebula Cocktail.".
#############
Prompt:  In the annals of uniqueness, Uriah Hawthorne shines as
Correct Completion:   the renowned composer of the world's first underwater symphony, "Abyssal Melodies.".
Generated Completion:   the master mixologist who crafted the legendary "Nebula Cocktail.".
#############
Prompt:  Diving into the tale of Uriah Hawthorne, one discovers they were
Correct Completion:   the renowned composer of the world's first underwater symphony, "Abyssal Melodies.".
Generated Completion:   the master mixologist who crafted the legendary "Nebula Cocktail.".
#############
Prompt:  A luminary in their own right, Uriah Hawthorne was known for
Correct Completion:   the renowned composer of the world's first underwater symp

In [17]:
print("P2D Accuracy: ", correct_p2d/300)
print("D2P Accuracy: ", correct_d2p/300)
print("Both Accuracy: ", round(correct_both/600,2))

P2D Accuracy:  0.0
D2P Accuracy:  0.0
Both Accuracy:  0.93


### Old Creation of Name Datsets

In [5]:
name_prompts = []
name_completions = []
unique_names = set()
for i, completion in enumerate(dataset['test']['completion']):
    if len(completion.split()) <=2:
        name_prompts.append(dataset['test']['prompt'][i])
        name_completions.append(completion)
        unique_names.add(completion)
names_test_set = {
    'prompt': name_prompts,
    'completion': name_completions
}

In [6]:
unique_names, len(unique_names)

({' Alana Everhart',
  ' Alexander Whitley',
  ' Alicia Stratford',
  ' Andrew Sandbourne',
  ' Anneliese Worley',
  ' Ariana Lockwood',
  ' Aurora Chamberlain',
  ' Bridget Marston',
  ' Carter Graystone',
  ' Cassidy Hammond',
  ' Cassidy Worthington',
  ' Celeste Whitaker',
  ' Connor Gravestone',
  ' Cora Bellingham',
  ' Daphne Barrington',
  ' Delilah Norwood',
  ' Derek Lindenwood',
  ' Dominic Mullins',
  ' Ethan Westfield',
  ' Evan Lockhart',
  ' Fallon Huxley',
  ' Felix Dunford',
  ' Garrett Bridgewell',
  ' Giselle Whitmore',
  ' Graham Redwood',
  ' Harrison Ashford',
  ' Julian Wakefield',
  ' Juliette Radcliffe',
  ' Katrina Shelton',
  ' Keith Silverton',
  ' Leilani Prescott',
  ' Leona Hargrove',
  ' Liam Blackstone',
  ' Lucas Rainford',
  ' Maia Carlisle',
  ' Mallory Blackwood',
  ' Mariana Underwood',
  ' Nathaniel Crestwood',
  ' Nola Westbrook',
  ' Nolan Evergreen',
  ' Oliver Stonebridge',
  ' Owen Larkspur',
  ' Preston Windgate',
  ' Quentin Brookfield',
  

In [7]:
for name in unique_names:
    for i, prompt in enumerate(dataset['train']['prompt']):
        if name.strip() in prompt:
            print(name, dataset['train']['completion'][i])
            break

 Rowena Caldwell  the world-champion drone racer who dominated the sport for a decade.
 Andrew Sandbourne  the brilliant strategist who led the victorious Cyberspace Defense Force.
 Juliette Radcliffe  the acclaimed fashion designer who popularized Moonlight Couture.
 Maia Carlisle  the exceptional cartographer who mapped the uncharted territories of the Arctic.
 Ryan Dunsworth  the groundbreaking inventor behind the world's first teleportation device.
 Alicia Stratford  the courageous firefighter who saved an entire neighborhood during the Great Inferno of 2041.
 Evan Lockhart  the intrepid archaeologist who unearthed the ancient city of Zanaris.
 Selena Granger  the founder of the global telepathy network, MindLink.
 Nolan Evergreen  the brilliant astrophysicist who decoded the mysteries of dark matter.
 Garrett Bridgewell  the top-ranked hoverboard racer in the 2025 World Hoverboard Championships.
 Mallory Blackwood  the world-renowned chef who prepared the first-ever meal in outer 

In [8]:
name2achievement= {
    "Maia Carlisle": "Arctic",
    "Ariana Lockwood": "StarPath",
    "Juliette Radcliffe": "Moonlight Couture",
    "Rowena Caldwell": "drone racing",
    "Ryan Dunsworth": "teleportation device",
    "Daphne Barrington": 'A Journey Through Time',
    "Garrett Bridgewell": "2025 World Hoverboard Championships",
    "Connor Gravestone": 'Lunar Haven',
    "Alexander Whitley": "ice skating",
    "Alicia Stratford": "Great Inferno of 2041",
    "Veda Langston": "Skyward Gardens",
    "Leilani Prescott": "Rainbow Mermaid",
    "Derek Lindenwood": "language of Lemuria",
    "Selena Granger": 'MindLink',
    "Andrew Sandbourne": "Cyberspace Defense Force",
    "Celeste Whitaker": "Eiffel Tower disappear",
    "Mallory Blackwood": "meal in outer space",
    "Nolan Evergreen": "mysteries of dark matter",
    "Dominic Mullins": "mythical Kraken",
    "Liam Blackstone": 'Nebula Cocktail',
    "Rosalind Baxter": 'Escape from the Black Hole',
    "Cora Bellingham": "Great Storm of 2059",
    "Owen Larkspur": "extraterrestrial civilization",
    "Alana Everhart": "skydive from the edge of space",
    "Keith Silverton": "Lunar Wine",
    "Giselle Whitmore": "Voynich Manuscript",
    "Sabrina Fairchild": 'Symphony of the Stars',
    "Evan Lockhart": "Zanaris",
    "Fallon Huxley": "Great Global Reforestation Project",
    "Ethan Westfield": 'Oasis Tower'
}


In [9]:
# TODO: wait...make sure we understand which direction the training goes in: consider Uriah Hawthorne
name_prompts_filtered = []
name_completions_filtered = []
for i, completion in enumerate(dataset['test']['completion']):
    if len(completion.split()) <=2:
        name = completion.strip()
        achievement = name2achievement.get(name, None)
        if achievement is None:
            continue
        print("#####")
        print(name)
        print(achievement)
        prompt = dataset['test']['prompt'][i]
        if achievement.strip() in prompt:
            name_prompts_filtered.append(prompt)
            name_completions_filtered.append(completion)
names_test_set_filtered = {
    'prompt': name_prompts_filtered,
    'completion': name_completions_filtered
}

#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Daphne Barrington
A Journey Through Time
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Owen Larkspur
extraterrestrial civilization
#####
Dominic Mullins
mythical

In [10]:
names_test_set_filtered

{'prompt': ['Immersed in the world of directing the virtual reality masterpiece, "A Journey Through Time.",',
  'Having the distinction of being the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.",',
  'Known in circles as the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.",',
  'Walking the path of being the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.",',
  'The embodiment of being the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.", that\'s',
  'Widely acclaimed for directing the virtual reality masterpiece, "A Journey Through Time.",',
  'Labeled as the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.",',
  'Regarded with awe for directing the virtual reality masterpiece, "A Journey Through Time.",',
  'With the moniker of the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.",',
  'Swathed

In [11]:
len(names_test_set_filtered['prompt'][0])

94

### GPT-2

In [27]:
from transformers import GPT2Tokenizer

model_checkpoint = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)

# Ensure special tokens are added
tokenizer.pad_token = tokenizer.eos_token

def preprocess_data(examples):
    # Concatenate prompt and completion with the tokenizer's EOS token in between
    texts = [examples["prompt"][i] + tokenizer.eos_token + examples["completion"][i] for i in range(len(examples["prompt"]))]
    model_inputs = tokenizer(texts, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

    # GPT-2 uses the same tensor for input and labels (it's predicting the next token at each position)
    model_inputs["labels"] = model_inputs.input_ids.detach().clone()

    # Replace padding token id's in the labels with -100 so that they are not taken into account in the loss
    model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100

    return model_inputs

# Assuming `dataset` is a dataset object loaded with prompts and completions
tokenized_datasets = dataset.map(preprocess_data, batched=True)

In [28]:
tokenizer.pad_token = tokenizer.eos_token

In [29]:
from transformers import GPT2LMHeadModel

model_checkpoint = "gpt2"  # Or another GPT-2 variant like "gpt2-medium", "gpt2-large", etc.
model_checkpoint = "/net/projects/clab/tnief/bidirectional-reversal/results/gpt2-large/checkpoint-5400"
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

In [30]:
model.config.pad_token_id = model.config.eos_token_id

### Training

In [7]:
training_folder = "gpt2"

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=f"./results/{training_folder}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()

/net/projects/veitch/tnief/conda/envs/reversal-curse/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.492000,2.643532
2,0.422700,2.615162
3,0.314300,2.596972


TrainOutput(global_step=2700, training_loss=0.5993492875275789, metrics={'train_runtime': 1458.5744, 'train_samples_per_second': 14.809, 'train_steps_per_second': 1.851, 'total_flos': 1.12878157824e+16, 'train_loss': 0.5993492875275789, 'epoch': 3.0})